In [5]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import requests
from io import BytesIO
import numpy as np

class SceneRecognizer:
    def __init__(self, model_type='resnet50'):
        """
        Инициализация распознавателя сцены

        Args:
            model_type: тип модели (resnet50, efficientnet, etc.)
        """
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model_type = model_type
        self.model = None
        self.labels = [
            'airport_inside', 'auditorium', 'bakery', 'bar', 'bedroom',
            'bookstore', 'bowling', 'buffet', 'casino', 'children_room',
            'church_inside', 'classroom', 'closet', 'clothingstore', 'computerroom',
            'concert_hall', 'dining_room', 'elevator', 'fastfood_restaurant',
            'florist', 'gameroom', 'garage', 'greenhouse', 'gym',
            'hospitalroom', 'inside_bus', 'inside_subway', 'jewelleryshop',
            'kitchen', 'library', 'livingroom', 'mall', 'meeting_room',
            'movietheater', 'museum', 'nursery', 'office', 'parking',
            'pharmacy', 'restaurant', 'shoeshop', 'stairscase', 'studiomusic',
            'subway', 'toystore', 'trainstation', 'tv_studio', 'videostore'
        ]

        self._load_model()
        self._setup_transforms()

    def _load_model(self):
        """Загрузка предобученной модели"""
        if self.model_type == 'resnet50':
            self.model = models.resnet50(pretrained=True)
            # Заменяем последний слой для нашего числа классов
            num_features = self.model.fc.in_features
            self.model.fc = nn.Linear(num_features, len(self.labels))
        else:
            raise ValueError(f"Модель {self.model_type} не поддерживается")

        self.model = self.model.to(self.device)
        self.model.eval()

    def _setup_transforms(self):
        """Настройка преобразований для изображения"""
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])

    def predict(self, image_path_or_url):
        """
        Предсказание сцены на изображении

        Args:
            image_path_or_url: путь к файлу или URL изображения

        Returns:
            dict: результаты предсказания
        """
        # Загрузка изображения
        if image_path_or_url.startswith('http'):
            response = requests.get(image_path_or_url)
            image = Image.open(BytesIO(response.content)).convert('RGB')
        else:
            image = Image.open(image_path_or_url).convert('RGB')

        # Преобразование изображения
        input_tensor = self.transform(image).unsqueeze(0).to(self.device)

        # Предсказание
        with torch.no_grad():
            outputs = self.model(input_tensor)
            probabilities = torch.nn.functional.softmax(outputs[0], dim=0)
            top_probs, top_indices = torch.topk(probabilities, 5)

        # Формирование результатов
        results = []
        for i in range(len(top_indices)):
            results.append({
                'scene': self.labels[top_indices[i].item()],
                'confidence': top_probs[i].item()
            })

        return {
            'top_predictions': results,
            'primary_scene': results[0]['scene'],
            'primary_confidence': results[0]['confidence']
        }

    def predict_batch(self, image_paths):
        """Пакетное предсказание для нескольких изображений"""
        results = []
        for path in image_paths:
            try:
                result = self.predict(path)
                results.append(result)
            except Exception as e:
                results.append({'error': str(e)})
        return results

# Демонстрация работы
def demo():
    """Демонстрация работы распознавателя сцены"""
    recognizer = SceneRecognizer()

    # Примеры изображений (можно заменить на реальные пути или URL)
    test_images = [
        '/content/images (1).jfif',  # заменить на реальный URL
        '/content/organizaciya_prostransva_ofisa_06.jpg',   # заменить на реальный URL
        '/content/organizaciya_prostransva_ofisa_06.jpg'           # локальный файл
    ]

    for i, img_path in enumerate(test_images):
        print(f"\n--- Анализ изображения {i+1} ---")
        try:
            result = recognizer.predict(img_path)
            print(f"Основная сцена: {result['primary_scene']}")
            print(f"Уверенность: {result['primary_confidence']:.4f}")
            print("Топ-5 предсказаний:")
            for pred in result['top_predictions']:
                print(f"  {pred['scene']}: {pred['confidence']:.4f}")
        except Exception as e:
            print(f"Ошибка при обработке изображения: {e}")

# Альтернативная версия с использованием предобученных весов Places365
class Places365SceneRecognizer:
    def __init__(self):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self._load_places365_model()
        self._setup_transforms()

    def _load_places365_model(self):
        """Загрузка модели, обученной на датасете Places365"""
        self.model = models.resnet50(pretrained=False)

        # Загружаем веса, обученные на Places365
        try:
            # В реальном проекте здесь будет загрузка предобученных весов
            model_path = 'resnet50_places365.pth.tar'
            checkpoint = torch.load(model_path, map_location=self.device)
            state_dict = {str.replace(k, 'module.', ''): v for k, v in checkpoint['state_dict'].items()}
            self.model.load_state_dict(state_dict)
        except:
            print("Не удалось загрузить веса Places365, используем стандартные")
            self.model = models.resnet50(pretrained=True)

        self.model = self.model.to(self.device)
        self.model.eval()

        # Загружаем метки классов Places365
        self.labels = self._load_places365_labels()

    def _load_places365_labels(self):
        """Загрузка меток классов Places365"""
        # В реальном проекте загружаем из файла
        return [
            'abbey', 'airport_terminal', 'alley', 'amphitheater', 'apartment_building',
            'aquarium', 'arch', 'art_gallery', 'auditorium', 'badlands',
            # ... и так 365 классов
        ]

    def _setup_transforms(self):
        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])

    def predict(self, image_path):
        """Аналогично предыдущему методу"""
        # Реализация аналогична SceneRecognizer.predict()
        pass

if __name__ == "__main__":
    # Создаем простой тест с случайными данными
    recognizer = SceneRecognizer()

    # Создаем случайное изображение для демонстрации
    dummy_image = torch.randn(3, 224, 224)
    transforms.ToPILImage()(dummy_image).save('test_dummy.jpg')

    result = recognizer.predict('/content/organizaciya_prostransva_ofisa_06.jpg')
    print("Результат для тестового изображения:")
    print(f"Основная сцена: {result['primary_scene']}")
    print(f"Уверенность: {result['primary_confidence']:.4f}")

Результат для тестового изображения:
Основная сцена: videostore
Уверенность: 0.0631


In [6]:
demo()


--- Анализ изображения 1 ---
Основная сцена: bedroom
Уверенность: 0.0367
Топ-5 предсказаний:
  bedroom: 0.0367
  computerroom: 0.0362
  inside_bus: 0.0359
  livingroom: 0.0354
  jewelleryshop: 0.0348

--- Анализ изображения 2 ---
Основная сцена: livingroom
Уверенность: 0.0623
Топ-5 предсказаний:
  livingroom: 0.0623
  computerroom: 0.0559
  dining_room: 0.0398
  museum: 0.0337
  bedroom: 0.0315

--- Анализ изображения 3 ---
Основная сцена: livingroom
Уверенность: 0.0623
Топ-5 предсказаний:
  livingroom: 0.0623
  computerroom: 0.0559
  dining_room: 0.0398
  museum: 0.0337
  bedroom: 0.0315


In [7]:
import torch
from torchvision import models, transforms
from PIL import Image
import json

# Простая версия с использованием предобученной модели
def simple_scene_recognition(image_path):
    """
    Простое распознавание сцены с использованием ImageNet весов
    (для демонстрационных целей)
    """
    # Загрузка модели
    model = models.resnet50(pretrained=True)
    model.eval()

    # Преобразования
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])

    # Загрузка и преобразование изображения
    image = Image.open(image_path).convert('RGB')
    input_tensor = transform(image).unsqueeze(0)

    # Предсказание
    with torch.no_grad():
        output = model(input_tensor)

    # Получение результатов
    probabilities = torch.nn.functional.softmax(output[0], dim=0)

    # Загрузка меток ImageNet (в реальном проекте нужны метки сцен)
    with open('imagenet_classes.txt') as f:
        classes = [line.strip() for line in f.readlines()]

    # Топ-5 предсказаний
    top5_prob, top5_catid = torch.topk(probabilities, 5)

    results = []
    for i in range(top5_prob.size(0)):
        results.append({
            'class': classes[top5_catid[i]],
            'confidence': top5_prob[i].item()
        })

    return results

# Пример использования
if __name__ == "__main__":
    results = simple_scene_recognition('/content/kuhnya-eterno-790-1.jpg')
    for result in results:
        print(f"{result['class']}: {result['confidence']:.4f}")

FileNotFoundError: [Errno 2] No such file or directory: 'imagenet_classes.txt'

In [8]:
import torch
from torchvision import models, transforms
from PIL import Image
import requests

def simple_scene_recognition(image_path):
    """
    Простое распознавание сцены с использованием ImageNet весов
    """
    # Загрузка модели
    model = models.resnet50(pretrained=True)
    model.eval()

    # Преобразования
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])

    # Загрузка и преобразование изображения
    image = Image.open(image_path).convert('RGB')
    input_tensor = transform(image).unsqueeze(0)

    # Предсказание
    with torch.no_grad():
        output = model(input_tensor)

    # Получение результатов
    probabilities = torch.nn.functional.softmax(output[0], dim=0)

    # Загрузка меток ImageNet из интернета
    classes = load_imagenet_labels()

    # Топ-5 предсказаний
    top5_prob, top5_catid = torch.topk(probabilities, 5)

    results = []
    for i in range(top5_prob.size(0)):
        results.append({
            'class': classes[top5_catid[i]],
            'confidence': top5_prob[i].item()
        })

    return results

def load_imagenet_labels():
    """Загрузка меток ImageNet"""
    # URL с метками ImageNet
    url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"

    try:
        response = requests.get(url)
        response.raise_for_status()
        classes = [line.strip() for line in response.text.split('\n') if line.strip()]
        return classes
    except:
        # Если не удалось загрузить, создаем базовые метки
        return [f"class_{i}" for i in range(1000)]

# Пример использования
if __name__ == "__main__":
    results = simple_scene_recognition('/content/kuhnya-eterno-790-1.jpg')
    print("Топ-5 предсказаний:")
    for i, result in enumerate(results, 1):
        print(f"{i}. {result['class']}: {result['confidence']:.4f}")

Топ-5 предсказаний:
1. home theater: 0.3208
2. dishwasher: 0.1524
3. microwave: 0.1418
4. stove: 0.0847
5. washbasin: 0.0693
